In [4]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import datetime  # For datetime objects
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])

# Import the backtrader platform
import backtrader as bt
import backtrader.indicators as btind

from backtrader.feeds import GenericCSVData

from statistics import mean
from statistics import median

In [5]:
class dix_csv(GenericCSVData):
    
    lines= ('dix', 'gex')
    
    params = (
        ('dtformat', '%m/%d/%Y'),
        ('datetime', 0),
        ('time', -1),
        ('open', 1),
        ('high', 2),
        ('low', 3),
        ('close', 4),
        ('volume', 6),
        ('openinterest', -1),
        ('dix', 9),
        ('gex', 10),        
    )

In [6]:
#Optimize DIX MA
class DIXmaopt(bt.Strategy):
    params = (
        ('period', 2),
        ('printlog', False),
        ('dixupper', 45),
        ('dixlower', 40),
    )
    def start(self):
        self.val_start = self.broker.get_cash()  # keep the starting cash

    def log(self, txt, dt=None, doprint=False):
        ''' Logging function fot this strategy'''
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close

        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None

        #Add a DIXSMA Indicator
        self.dixsma = bt.indicators.MovingAverageSimple(
           self.data.dix, period=self.params.period)


    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))
        #make a list of the pnl for each trade
        self.pnl_percent = trade.pnl / self.broker.getvalue()
        self.pnl_list=[]
        self.pnl_list.append(self.pnl_percent)

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:

            # buy when dix greater than dixupper threshold
            if self.dixsma >= (self.params.dixupper/100):
                self.log('BUY CREATE, %.2f' % self.dataclose[0])
                self.order = self.buy()
                
        else:
            
            # sell after 60 trading days
            self.duration = len(self) - self.bar_executed + 1
            if self.duration >= 60:
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()
    def stop(self):
        #roi
        self.roi = (self.broker.get_value() / self.val_start) - 1.0
        #take the mean if a pnl exists
        try:
            self.pnl_mean = (mean(self.pnl_list) * 100)
        except:
            self.pnl_list = [0]
            self.pnl_mean = (mean(self.pnl_list) * 100)
        #take the median if a pnl exists    
        try:
            self.pnl_median = (median(self.pnl_list) * 100)
        except:
            self.pnl_list = [0]
            self.pnl_median = (median(self.pnl_list) * 100)
            
        print('(MA Period%2d) (DIXUpper %2d) (MEAN TRADE PNL: %.3f%%) (MEDIAN TRADE PNL: %.3f%%) ROI %.2f%%' %
                 (self.params.period, self.params.dixupper, self.pnl_mean, self.pnl_median, self.roi))
        
        
#runDIXma Strategy Opt
if __name__ == '__main__':
    # Create a cerebro entity
    cerebro = bt.Cerebro()    
    
    # Add a strategy
    strats = cerebro.optstrategy(
        DIXmaopt,
        period=range(1,501)
    ) 
    
    
    # Datas are in a subfolder of the samples. Need to find where the script is
    # because it could have been called from anywhere
    modpath = os.path.dirname(os.path.abspath(sys.argv[0]))
    datapath = os.path.join(modpath, 'C:\\Users\\big_u\\DIX MA Testing Backtrader\DIX OHLC.csv')

    # Create a Data Feed
    data = dix_csv(
        dataname=datapath,
    )

    # Add the Data Feed to Cerebro
    cerebro.adddata(data)

    # Set our desired cash start
    cerebro.broker.setcash(100000.0)

    # Add a FixedSize sizer according to the stake
    cerebro.addsizer(bt.sizers.FixedSize, stake=1)

    # Print out the starting conditions
    #print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run over everything
    cerebro.run(maxcpus=1)
    
    # Set the commission
    cerebro.broker.setcommission(commission=0.00)

    # Print out the final result
    #print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())


(MA Period 1) (DIXUpper 45) (MEAN TRADE PNL: 0.583%) (MEDIAN TRADE PNL: 0.583%) ROI 0.02%
(MA Period 2) (DIXUpper 45) (MEAN TRADE PNL: 0.575%) (MEDIAN TRADE PNL: 0.575%) ROI 0.01%
(MA Period 3) (DIXUpper 45) (MEAN TRADE PNL: 0.550%) (MEDIAN TRADE PNL: 0.550%) ROI 0.01%
(MA Period 4) (DIXUpper 45) (MEAN TRADE PNL: 0.551%) (MEDIAN TRADE PNL: 0.551%) ROI 0.01%
(MA Period 5) (DIXUpper 45) (MEAN TRADE PNL: 0.429%) (MEDIAN TRADE PNL: 0.429%) ROI 0.01%
(MA Period 6) (DIXUpper 45) (MEAN TRADE PNL: 0.570%) (MEDIAN TRADE PNL: 0.570%) ROI 0.01%
(MA Period 7) (DIXUpper 45) (MEAN TRADE PNL: 0.569%) (MEDIAN TRADE PNL: 0.569%) ROI 0.01%
(MA Period 8) (DIXUpper 45) (MEAN TRADE PNL: 0.554%) (MEDIAN TRADE PNL: 0.554%) ROI 0.01%
(MA Period 9) (DIXUpper 45) (MEAN TRADE PNL: 0.550%) (MEDIAN TRADE PNL: 0.550%) ROI 0.02%
(MA Period10) (DIXUpper 45) (MEAN TRADE PNL: 0.526%) (MEDIAN TRADE PNL: 0.526%) ROI 0.02%
(MA Period11) (DIXUpper 45) (MEAN TRADE PNL: 0.518%) (MEDIAN TRADE PNL: 0.518%) ROI 0.02%
(MA Period

(MA Period92) (DIXUpper 45) (MEAN TRADE PNL: 0.075%) (MEDIAN TRADE PNL: 0.075%) ROI 0.00%
(MA Period93) (DIXUpper 45) (MEAN TRADE PNL: 0.069%) (MEDIAN TRADE PNL: 0.069%) ROI 0.00%
(MA Period94) (DIXUpper 45) (MEAN TRADE PNL: 0.062%) (MEDIAN TRADE PNL: 0.062%) ROI 0.00%
(MA Period95) (DIXUpper 45) (MEAN TRADE PNL: 0.065%) (MEDIAN TRADE PNL: 0.065%) ROI 0.00%
(MA Period96) (DIXUpper 45) (MEAN TRADE PNL: 0.094%) (MEDIAN TRADE PNL: 0.094%) ROI 0.00%
(MA Period97) (DIXUpper 45) (MEAN TRADE PNL: 0.074%) (MEDIAN TRADE PNL: 0.074%) ROI 0.00%
(MA Period98) (DIXUpper 45) (MEAN TRADE PNL: 0.063%) (MEDIAN TRADE PNL: 0.063%) ROI 0.00%
(MA Period99) (DIXUpper 45) (MEAN TRADE PNL: 0.073%) (MEDIAN TRADE PNL: 0.073%) ROI 0.00%
(MA Period100) (DIXUpper 45) (MEAN TRADE PNL: 0.083%) (MEDIAN TRADE PNL: 0.083%) ROI 0.00%
(MA Period101) (DIXUpper 45) (MEAN TRADE PNL: 0.117%) (MEDIAN TRADE PNL: 0.117%) ROI 0.00%
(MA Period102) (DIXUpper 45) (MEAN TRADE PNL: 0.117%) (MEDIAN TRADE PNL: 0.117%) ROI 0.00%
(MA Per

(MA Period182) (DIXUpper 45) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.00%
(MA Period183) (DIXUpper 45) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.00%
(MA Period184) (DIXUpper 45) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.00%
(MA Period185) (DIXUpper 45) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.00%
(MA Period186) (DIXUpper 45) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.00%
(MA Period187) (DIXUpper 45) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.00%
(MA Period188) (DIXUpper 45) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.00%
(MA Period189) (DIXUpper 45) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.00%
(MA Period190) (DIXUpper 45) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.00%
(MA Period191) (DIXUpper 45) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.00%
(MA Period192) (DIXUpper 45) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.00%

(MA Period273) (DIXUpper 45) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.00%
(MA Period274) (DIXUpper 45) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.00%
(MA Period275) (DIXUpper 45) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.00%
(MA Period276) (DIXUpper 45) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.00%
(MA Period277) (DIXUpper 45) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.00%
(MA Period278) (DIXUpper 45) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.00%
(MA Period279) (DIXUpper 45) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.00%
(MA Period280) (DIXUpper 45) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.00%
(MA Period281) (DIXUpper 45) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.00%
(MA Period282) (DIXUpper 45) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.00%
(MA Period283) (DIXUpper 45) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.00%

(MA Period364) (DIXUpper 45) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.00%
(MA Period365) (DIXUpper 45) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.00%
(MA Period366) (DIXUpper 45) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.00%
(MA Period367) (DIXUpper 45) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.00%
(MA Period368) (DIXUpper 45) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.00%
(MA Period369) (DIXUpper 45) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.00%
(MA Period370) (DIXUpper 45) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.00%
(MA Period371) (DIXUpper 45) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.00%
(MA Period372) (DIXUpper 45) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.00%
(MA Period373) (DIXUpper 45) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.00%
(MA Period374) (DIXUpper 45) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.00%

(MA Period455) (DIXUpper 45) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.00%
(MA Period456) (DIXUpper 45) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.00%
(MA Period457) (DIXUpper 45) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.00%
(MA Period458) (DIXUpper 45) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.00%
(MA Period459) (DIXUpper 45) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.00%
(MA Period460) (DIXUpper 45) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.00%
(MA Period461) (DIXUpper 45) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.00%
(MA Period462) (DIXUpper 45) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.00%
(MA Period463) (DIXUpper 45) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.00%
(MA Period464) (DIXUpper 45) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.00%
(MA Period465) (DIXUpper 45) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.00%